In [1]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Authentication
# Create variable
# file_name = 'keys.py'
# %%writefile {file_name}

# consumer_key="insert your API key"
# consumer_secret="insert your API secret"
# access_token="insert your access token"
# access_token_secret="insert your access token secret"
# bearer_token="insert your bearer token"
consumerKey = "7WRzB3IWiSVmJ7Q5uG0lPEPE0"
consumerSecret = "lPCyQvfWFtq6Z5Ek89ohQCb7nFrbbl76RBHw1mSdIUu2HCHO0U"
accessToken = "934117903791788032-aoGmzlY1NSfJH1WaycL88w3mnDHFwXA"
accessTokenSecret = "4PbK1Q7dr29CLSuZBVnW6OfNbdyUa26fZEPo7vYHBN2Ch"
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)
client = tweepy.Client(bearer_token = "AAAAAAAAAAAAAAAAAAAAAM%2BubgEAAAAANYJCmdOM9g7oNngHBtRo%2FSl8AkI%3Dkx2ri4hqIZ91I74hd2RR1hvkcsOegT70qU8SAqVAoSeuLYutGQ")


In [124]:
tweets = client.search_all_tweets(query = "#TheBachelorette -is:retweet",tweet_fields=['author_id', 'created_at'], max_results=500)


In [125]:
import pandas as pd
import json


In [126]:
all_tweets = []

for tweet in tweets.data:
    #print('\n'.join(dir(tweet)))
    all_tweets.append( [tweet.text, tweet.created_at] )

In [127]:
df = pd.DataFrame(all_tweets, columns=['Tweets', 'Created At'])
df.head()

,Tweets,Created At
0,When you’re absolutely buzzin for 51 days time...,2022-05-21 22:40:04+00:00
1,We’re in for the adventure of a lifetime. #The...,2022-05-21 22:00:00+00:00
2,IM SO READY #TheBachelorette https://t.co/mUDQ...,2022-05-21 21:17:48+00:00
3,The cutest promo ever! Ugh I just love Gabby &...,2022-05-21 17:59:55+00:00
4,Thinking about the Andrew and Greg pipeline fr...,2022-05-21 16:35:11+00:00


In [128]:
stopwordlist = ['a', 'about', 'above', 'after', 'again', 'ain', 'all', 'am', 'an',
             'and','any','are', 'as', 'at', 'be', 'because', 'been', 'before',
             'being', 'below', 'between','both', 'by', 'can', 'd', 'did', 'do',
             'does', 'doing', 'down', 'during', 'each','few', 'for', 'from',
             'further', 'had', 'has', 'have', 'having', 'he', 'her', 'here',
             'hers', 'herself', 'him', 'himself', 'his', 'how', 'i', 'if', 'in',
             'into','is', 'it', 'its', 'itself', 'just', 'll', 'm', 'ma',
             'me', 'more', 'most','my', 'myself', 'now', 'o', 'of', 'on', 'once',
             'only', 'or', 'other', 'our', 'ours','ourselves', 'out', 'own', 're','s', 'same', 'she', "shes", 'should', "shouldve",'so', 'some', 'such',
             't', 'than', 'that', "thatll", 'the', 'their', 'theirs', 'them',
             'themselves', 'then', 'there', 'these', 'they', 'this', 'those',
             'through', 'to', 'too','under', 'until', 'up', 've', 'very', 'was',
             'we', 'were', 'what', 'when', 'where','which','while', 'who', 'whom',
             'why', 'will', 'with', 'won', 'y', 'you', "youd","youll", "youre",
             "youve", 'your', 'yours', 'yourself', 'yourselves']
STOPWORDS = set(stopwordlist)
def cleaning_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])
df['Tweets'] = df['Tweets'].apply(lambda text: cleaning_stopwords(text))
df['Tweets'].head()

0    When you’re absolutely buzzin 51 days time bra...
1    We’re adventure lifetime. #TheBachelorette ret...
2    IM SO READY #TheBachelorette https://t.co/mUDQ...
3    The cutest promo ever! Ugh I love Gabby &amp; ...
4    Thinking Andrew Greg pipeline bachelorette con...
Name: Tweets, dtype: object

In [129]:
import string
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def cleaning_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_punctuations(x))
df['Tweets'].tail()
def cleaning_repeating_char(text):
    return re.sub(r'(.)1+', r'1', text)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_repeating_char(x))
df['Tweets'].tail()
def cleaning_URLs(data):
    return re.sub('((www.[^s]+)|(https?://[^s]+))',' ',data)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_URLs(x))
df['Tweets'].tail()

def cleaning_numbers(data):
    return re.sub('[0-9]+', '', data)
df['Tweets'] = df['Tweets'].apply(lambda x: cleaning_numbers(x))
df['Tweets'].head()

0    When you’re absolutely buzzin  days time brand...
1    We’re adventure lifetime TheBachelorette retur...
2        IM SO READY TheBachelorette httpstcomUDQGodLu
3    The cutest promo ever Ugh I love Gabby amp Rac...
4    Thinking Andrew Greg pipeline bachelorette con...
Name: Tweets, dtype: object

In [130]:
from nltk.sentiment import SentimentIntensityAnalyzer
import operator
sia = SentimentIntensityAnalyzer()
df["sentiment_score"] = df["Tweets"].apply(lambda x: sia.polarity_scores(x)["compound"])
df["sentiment"] = np.select([df["sentiment_score"] < 0, df["sentiment_score"] == 0, df["sentiment_score"] > 0],
                           ['neg', 'neu', 'pos'])

In [131]:
df.head()

,Tweets,Created At,sentiment_score,sentiment
0,When you’re absolutely buzzin days time brand...,2022-05-21 22:40:04+00:00,0.4201,pos
1,We’re adventure lifetime TheBachelorette retur...,2022-05-21 22:00:00+00:00,0.3182,pos
2,IM SO READY TheBachelorette httpstcomUDQGodLu,2022-05-21 21:17:48+00:00,0.6439,pos
3,The cutest promo ever Ugh I love Gabby amp Rac...,2022-05-21 17:59:55+00:00,0.9670,pos
4,Thinking Andrew Greg pipeline bachelorette con...,2022-05-21 16:35:11+00:00,0.0000,neu


In [132]:
import plotly.express as px

In [133]:
df[df['sentiment'] == 'neg'].Tweets.head()

9     Why nice female Thai student go mad Bangkok wr...
10    Gabby Rachel’s first promo Katie level quality...
12    I know Demi ass throwing thee biggest tantrum ...
35    I’ve never excited bachelorette season IN MY L...
55    TheBachelorette alum JoJo Fletcher accused get...
Name: Tweets, dtype: object

In [134]:
fig = px.histogram(df, x="sentiment", nbins=3)
fig.show()

In [135]:
from textblob import TextBlob
df["sentiment_scoreTB"] = df["Tweets"].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df["sentimentTB"] = np.select([df["sentiment_scoreTB"] < 0, df["sentiment_scoreTB"] == 0, df["sentiment_scoreTB"] > 0],
                           ['neg', 'neu', 'pos'])

In [136]:
fig = px.histogram(df, x="sentimentTB", nbins=3)
fig.show()

In [137]:
from flair.models import TextClassifier
from flair.data import Sentence
sia = TextClassifier.load('en-sentiment')
def flair_prediction(x):
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    if "POSITIVE" in str(score):
        return "pos"
    elif "NEGATIVE" in str(score):
        return "neg"
    else:
        return "neu"
    
df["Flair_sentiment"] = df["Tweets"].apply(flair_prediction)


2022-05-21 19:59:51,184 loading file /home/kevin/.flair/models/sentiment-en-mix-distillbert_4.pt


In [138]:
r = Sentence(df['Tweets'][5])
sia.predict(r)
print(r)

Sentence: "Oh LFG corniest Bachelorette promo recent memory I cannot WAIT July TheBachelorette httpstcoHLqOSHKC" → NEGATIVE (0.6157)


In [139]:
fig = px.histogram(df, x="Flair_sentiment", nbins=3)
fig.show()

In [140]:
df[df['Flair_sentiment'] == 'neg'].Tweets.head()

5     Oh LFG corniest Bachelorette promo recent memo...
6     Check new TheBachelorette promo honors MeanGir...
9     Why nice female Thai student go mad Bangkok wr...
10    Gabby Rachel’s first promo Katie level quality...
11    TheBachelorette BacheloretteABC couple Michell...
Name: Tweets, dtype: object

In [141]:
from transformers import pipeline
sentiment_pipeline = pipeline("sentiment-analysis")
data = ["I love you", "I hate you"]
sentiment_pipeline(data)

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [142]:
specific_model = pipeline(model="cardiffnlp/twitter-roberta-base-sentiment-latest")
df['pipeline'] = df['Tweets'].apply(lambda x: specific_model(x)[0]['label'])

Downloading:   0%|          | 0.00/996 [00:00<?, ?B/s]

In [143]:
specific_model(df['Tweets'][112])

[{'label': 'Neutral', 'score': 0.8289735317230225}]

In [144]:
df['Tweets'][112]

'TheBachelorette stars JoelleFletcher JRodger hitched — details romantic California I dos 👰🤵 httpstcotnyhYLdJuf'

In [145]:
fig = px.histogram(df, x="pipeline", nbins=3)
fig.show()

In [35]:

for tweet in tweets.data:
    print(tweet.text)
    if len(tweet.context_annotations) > 0:
        print(tweet.context_annotations)

When you’re absolutely buzzin for 51 days time when a brand new season of @BacheloretteABC lands in our life and we’re once again left wanting so much more. I am ready for another wild ride #Bachelorette #BachelorNation #TheBachelorette https://t.co/AivbNaHozE
We’re in for the adventure of a lifetime. #TheBachelorette returns to @ABCNetwork July 11. 😍🌹 https://t.co/qs0MkCF6O5
IM SO READY #TheBachelorette https://t.co/mUDQG3odLu
The cutest promo ever! Ugh I just love Gabby &amp; Rachel &amp; their friendship!! I can’t wait to watch their journey to find love! Rooting for them!! Hope their friendship becomes even stronger!! #TheBachelorette https://t.co/NUxhsR2Heg
Thinking about the Andrew and Greg pipeline from bachelorette contestants to full on bromance besties 💓 #TheBachelor #TheBachelorette
Oh LFG this is the corniest Bachelorette promo in recent memory I cannot WAIT for July 11
 
#TheBachelorette https://t.co/HLqO7S2HKC
Check out how the new #TheBachelorette promo honors #MeanGirls

In [32]:
counts = client.get_recent_tweets_count(query="#TheBachelorette", granularity='day')
counts


Response(data=[{'end': '2022-05-15T00:00:00.000Z', 'start': '2022-05-14T22:37:12.000Z', 'tweet_count': 0}, {'end': '2022-05-16T00:00:00.000Z', 'start': '2022-05-15T00:00:00.000Z', 'tweet_count': 35}, {'end': '2022-05-17T00:00:00.000Z', 'start': '2022-05-16T00:00:00.000Z', 'tweet_count': 66}, {'end': '2022-05-18T00:00:00.000Z', 'start': '2022-05-17T00:00:00.000Z', 'tweet_count': 29}, {'end': '2022-05-19T00:00:00.000Z', 'start': '2022-05-18T00:00:00.000Z', 'tweet_count': 20}, {'end': '2022-05-20T00:00:00.000Z', 'start': '2022-05-19T00:00:00.000Z', 'tweet_count': 19}, {'end': '2022-05-21T00:00:00.000Z', 'start': '2022-05-20T00:00:00.000Z', 'tweet_count': 223}, {'end': '2022-05-21T22:37:12.000Z', 'start': '2022-05-21T00:00:00.000Z', 'tweet_count': 182}], includes={}, errors=[], meta={'total_tweet_count': 574})

In [10]:
#Sentiment Analysis
def percentage(part,whole):
 return 100 * float(part)/float(whole)

noOfTweet = 100
tweets = #tweepy.Cursor(api.search, q=keyword).items(noOfTweet)
positive = 0
negative = 0
neutral = 0
polarity = 0
tweet_list = []
neutral_list = []
negative_list = []
positive_list = []
for tweet in tweets:
 
 #print(tweet.text)
 tweet_list.append(tweet.text)
 analysis = TextBlob(tweet.text)
 score = SentimentIntensityAnalyzer().polarity_scores(tweet.text)
 neg = score['neg']
 neu = score['neu']
 pos = score['pos']
 comp = score['compound']
 polarity += analysis.sentiment.polarity
 
if neg > pos:
 negative_list.append(tweet.text)
 negative += 1
elif pos > neg:
 positive_list.append(tweet.text)
 positive += 1
 
elif pos == neg:
 neutral_list.append(tweet.text)
 neutral += 1

positive = percentage(positive, noOfTweet)
negative = percentage(negative, noOfTweet)
neutral = percentage(neutral, noOfTweet)
polarity = percentage(polarity, noOfTweet)
positive = format(positive, '.1f')
negative = format(negative, '.1f')
neutral = format(neutral, '.1f')

AttributeError: 'API' object has no attribute 'search'